In [3]:
import pandas as pd
import numpy as np

In [5]:
import sys
sys.path.append("..")

from data import DATA_PATH

In [3]:
# risk-factors

risk_factors = None
for risk in (DATA_PATH / 'risk_factors').rglob('*.xlsx'):
    risk_name = risk.stem.lower()
    risk_data = pd.read_excel(risk)
    risk_data.columns = ['date', risk_name]
    if risk_factors is None:
        risk_factors = risk_data
    else:
        risk_factors = pd.merge(risk_factors, risk_data, how='left', on='date')

risk_factors = risk_factors.sort_values('date').dropna().reset_index(drop=True)

In [4]:
risk_factors.head(3)

,date,aluminum,brent,cbr_key_rate,eur_rub,moex_index,nickel,rtsi,usd_rub
0,2020-01-10,1806.0,65.02,6.25,68.0555,3123.66,14116.0,1614.69,61.2340
1,2020-01-13,1798.0,64.23,6.25,68.0450,3151.69,14007.5,1619.73,61.2632
2,2020-01-14,1809.0,64.42,6.25,67.8162,3129.77,13781.0,1604.96,60.9474


In [5]:
# stocks

stocks = None
for stock in (DATA_PATH / 'stocks').rglob('*.xlsx'):
    stock_name = stock.stem[:4].lower()
    stock_data = pd.read_excel(stock, header=1).rename(columns={'Дата': 'date', 'Legal Close': stock_name})[['date', stock_name]]
    if stocks is None:
        stocks = stock_data
    else:
        stocks = pd.merge(stocks, stock_data, how='left', on='date')

stocks = stocks.sort_values('date').reset_index(drop=True)

In [6]:
stocks.head(3)

,date,gazp,gmkn,lkoh,magn,mgnt,moex,rosn,rual,sber,vtbr
0,2020-01-03,259.00,194.48,6294.0,42.545,3445.5,107.18,456.9,30.605,255.00,0.046240
1,2020-01-06,256.55,195.88,6382.0,42.300,3416.0,108.03,455.6,30.260,253.90,0.046110
2,2020-01-08,255.79,198.50,6496.5,42.285,3427.0,110.84,452.0,30.155,259.15,0.046525


In [7]:
stocks[stocks.isna().any(axis=1)]

,date,gazp,gmkn,lkoh,magn,mgnt,moex,rosn,rual,sber,vtbr


In [8]:
# bonds

bonds = None
for bond in (DATA_PATH / 'bonds').rglob('*.csv'):
    bond_name = bond.stem[:7].lower()
    bond_data = pd.read_csv(bond).rename(columns={'<DATE>': 'date', '<CLOSE>': bond_name})[['date', bond_name]]
    if bonds is None:
        bonds = bond_data
    else:
        bonds = pd.merge(bonds, bond_data, how='left', on='date')

bonds['date'] = pd.to_datetime(bonds['date'], format='%d/%m/%y')
bonds = bonds.sort_values('date').reset_index(drop=True)

In [9]:
bonds.head(3)

,date,su26218,su26221,su26222,su26224,su26230
0,2020-01-03,118.49,111.500,104.851,105.311,113.205
1,2020-01-06,118.35,111.495,104.612,105.350,113.286
2,2020-01-08,118.10,111.374,104.842,105.011,113.290


In [10]:
bonds[bonds.isna().any(axis=1)]

,date,su26218,su26221,su26222,su26224,su26230


In [11]:
df = pd.merge(risk_factors, bonds, how='inner', on='date')
df = pd.merge(df, stocks, how='inner', on='date')

In [12]:
df.to_csv(DATA_PATH / 'all_data.csv')

### Add sofr

In [45]:
all_data = pd.read_csv(DATA_PATH / 'all_data.csv', index_col='date')
all_data.head()

,su26230_days_before_coupon,su26224_days_before_coupon,su26222_days_before_coupon,su26221_days_before_coupon,su26218_days_before_coupon,ecb_rate,aluminum,brent,cbr_key_rate,eur_rub,...,sber,vtbr,year_1,year_3,year_5,year_10,year_15,year_20,pca_cbd,sofr
date,,,,,,,,,,,,,,,,,,,,,
2020-01-10,89,145,103,89,82,0.25,1806.0,65.02,6.25,68.0555,...,258.19,0.04760,5.48,5.69,5.90,6.26,6.48,6.61,-5.601225,1.55
2020-01-11,89,145,103,89,82,0.25,1806.0,65.02,6.25,68.0555,...,258.19,0.04760,5.48,5.69,5.90,6.26,6.48,6.61,-5.601225,1.55
2020-01-12,89,145,103,89,82,0.25,1806.0,65.02,6.25,68.0555,...,258.19,0.04760,5.48,5.69,5.90,6.26,6.48,6.61,-5.601225,1.55
2020-01-13,86,142,100,86,79,0.25,1798.0,64.23,6.25,68.0450,...,262.40,0.04806,5.45,5.64,5.84,6.18,6.39,6.52,-5.760549,1.54
2020-01-14,85,141,99,85,78,0.25,1809.0,64.42,6.25,67.8162,...,259.05,0.04726,5.46,5.68,5.89,6.26,6.47,6.60,-5.626252,1.55


In [33]:
sofr_processed = (
    pd.read_excel(DATA_PATH / 'risk_factors/sofr.xlsx')
    .rename(columns={'Effective Date': 'date', 'Rate (%)': 'sofr'})
    .assign(date=lambda df: pd.to_datetime(df['date']))
    .set_index('date')
    .sort_index()
    .resample('1D')
    .ffill()
)

In [34]:
sofr_processed.index = sofr_processed.index.astype(str)

In [35]:
sofr_processed

,sofr
date,
2020-01-02,1.54
2020-01-03,1.55
2020-01-04,1.55
2020-01-05,1.55
2020-01-06,1.55
...,...
2024-06-14,5.31
2024-06-15,5.31
2024-06-16,5.31


In [36]:
sofr_processed.describe()

,sofr
count,1630.000000
mean,2.134368
std,2.279119
min,0.010000
25%,0.050000
50%,0.790000
75%,5.050000
max,5.400000


In [38]:
all_data = all_data.join(sofr_processed)

In [43]:
all_data.head()

,su26230_days_before_coupon,su26224_days_before_coupon,su26222_days_before_coupon,su26221_days_before_coupon,su26218_days_before_coupon,ecb_rate,aluminum,brent,cbr_key_rate,eur_rub,...,sber,vtbr,year_1,year_3,year_5,year_10,year_15,year_20,pca_cbd,sofr
date,,,,,,,,,,,,,,,,,,,,,
2020-01-10,89,145,103,89,82,0.25,1806.0,65.02,6.25,68.0555,...,258.19,0.04760,5.48,5.69,5.90,6.26,6.48,6.61,-5.601225,1.55
2020-01-11,89,145,103,89,82,0.25,1806.0,65.02,6.25,68.0555,...,258.19,0.04760,5.48,5.69,5.90,6.26,6.48,6.61,-5.601225,1.55
2020-01-12,89,145,103,89,82,0.25,1806.0,65.02,6.25,68.0555,...,258.19,0.04760,5.48,5.69,5.90,6.26,6.48,6.61,-5.601225,1.55
2020-01-13,86,142,100,86,79,0.25,1798.0,64.23,6.25,68.0450,...,262.40,0.04806,5.45,5.64,5.84,6.18,6.39,6.52,-5.760549,1.54
2020-01-14,85,141,99,85,78,0.25,1809.0,64.42,6.25,67.8162,...,259.05,0.04726,5.46,5.68,5.89,6.26,6.47,6.60,-5.626252,1.55


In [44]:
all_data.reset_index().to_csv(DATA_PATH / 'all_data.csv', index=False)